## A Test for Superior Predictive Ability

In [92]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import LinearRegression

In [93]:
df1 = pd.read_csv("C:/Users/cdbut/OneDrive/Documents/MyFin5350/Playground/Assignments/Data/crude-oil-combined.csv")
df2 = pd.read_csv("C:/Users/cdbut/OneDrive/Documents/MyFin5350/Playground/Assignments/Data/gasoline-combined.csv")
df3 = pd.read_csv("C:/Users/cdbut/OneDrive/Documents/MyFin5350/Playground/Assignments/Data/heating-oil-combined.csv")

#### Data Manipulation

In [94]:
df1 = df1.set_index("Date")
df2 = df2.set_index("Date")
df3 = df3.set_index("Date")

df1 = df1[["Futures"]]
df2 = df2[["Futures"]]
df3 = df3[["Futures"]]

df1.columns = ['F_CL']
df2.columns = ['F_GO']
df3.columns = ['F_HO']

df2 = df2 * 42
df3 = df3 * 42


In [95]:
df1 = df1.reindex(index=df1.index[::-1])
df2 = df2.reindex(index=df2.index[::-1])
df3 = df3.reindex(index=df3.index[::-1])

In [96]:
df_all = pd.merge(df1, df2, how="left", left_index=True, right_index=True)
df_all = pd.merge(df_all, df3, how="left", left_index=True, right_index=True)
df_all.tail()

,F_CL,F_GO,F_HO
Date,,,
1996-12-24,25.10,29.232,29.988
1996-12-26,24.92,29.190,29.442
1996-12-27,25.22,29.274,29.652
1996-12-30,25.37,29.232,29.652
1996-12-31,25.92,29.694,30.576


#### Create the Spread (Return)

In [97]:
df_all["sprd1"] = np.log(df_all["F_CL"]) - np.log(df_all["F_GO"])
df_all["sprd2"] = np.log(df_all["F_CL"]) - np.log(df_all["F_HO"])

In [98]:
df_all.tail()

,F_CL,F_GO,F_HO,sprd1,sprd2
Date,,,,,
1996-12-24,25.10,29.232,29.988,-0.152396,-0.177929
1996-12-26,24.92,29.190,29.442,-0.158155,-0.166752
1996-12-27,25.22,29.274,29.652,-0.149062,-0.161892
1996-12-30,25.37,29.232,29.652,-0.141697,-0.155962
1996-12-31,25.92,29.694,30.576,-0.135930,-0.165201


In [99]:
df = df_all[["sprd1", "sprd2"]]

In [100]:
df.tail()

,sprd1,sprd2
Date,,
1996-12-24,-0.152396,-0.177929
1996-12-26,-0.158155,-0.166752
1996-12-27,-0.149062,-0.161892
1996-12-30,-0.141697,-0.155962
1996-12-31,-0.135930,-0.165201


#### Create the Moving Averages
MA(4,1) , MA(8,1) , MA(12,1)

In [101]:
df["MAs1_12"] = df["sprd1"].rolling(window=60).mean()
df["MAs2_12"] = df["sprd2"].rolling(window=60).mean()
df["MAs1_8"] = df["sprd1"].rolling(window=40).mean()
df["MAs2_8"] = df["sprd2"].rolling(window=40).mean()
df["MAs1_4"] = df["sprd1"].rolling(window=20).mean()
df["MAs2_4"] = df["sprd2"].rolling(window=20).mean()
df["MAs1_1"] = df["sprd1"].rolling(window=5).mean()
df["MAs2_1"] = df["sprd2"].rolling(window=5).mean()


C:\Users\cdbut\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\cdbut\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [102]:
year = 5 # the number of years you want to look back
lookBack = year * 252
df = df.tail(lookBack)

In [103]:
df.tail()

,sprd1,sprd2,MAs1_12,MAs2_12,MAs1_8,MAs2_8,MAs1_4,MAs2_4,MAs1_1,MAs2_1
Date,,,,,,,,,,
1996-12-24,-0.152396,-0.177929,-0.139649,-0.205399,-0.157531,-0.206993,-0.148729,-0.201520,-0.144172,-0.178814
1996-12-26,-0.158155,-0.166752,-0.141000,-0.204704,-0.157174,-0.206428,-0.146865,-0.197090,-0.149051,-0.177158
1996-12-27,-0.149062,-0.161892,-0.142008,-0.203927,-0.156502,-0.205825,-0.144092,-0.193389,-0.155221,-0.178722
1996-12-30,-0.141697,-0.155962,-0.142937,-0.202710,-0.155261,-0.205482,-0.140937,-0.189108,-0.151205,-0.169736
1996-12-31,-0.135930,-0.165201,-0.143946,-0.201551,-0.154144,-0.205210,-0.140602,-0.186353,-0.147448,-0.165547


#### Generate signal variables

In [104]:
# if the signal > 0 then we have a "buy" signal
df["signal1"] = np.sign(df["MAs1_4"] - df["MAs1_1"]) # 1 = CL-GO MA(4,1)
df["signal2"] = np.sign(df["MAs2_4"] - df["MAs2_1"]) # 2 = CL-HO MA(4,1)
df["signal3"] = np.sign(df["MAs1_8"] - df["MAs1_1"]) # 3 = CL-GO MA(8,1)
df["signal4"] = np.sign(df["MAs2_8"] - df["MAs2_1"]) # 4 = CL-HO MA(8,1)
df["signal5"] = np.sign(df["MAs1_12"] - df["MAs1_1"]) # 5 = CL-GO MA(12,1)
df["signal6"] = np.sign(df["MAs2_12"] - df["MAs2_1"]) # 6 = CL-HO MA(12,1)
df.tail()

,sprd1,sprd2,MAs1_12,MAs2_12,MAs1_8,MAs2_8,MAs1_4,MAs2_4,MAs1_1,MAs2_1,signal1,signal2,signal3,signal4,signal5,signal6
Date,,,,,,,,,,,,,,,,
1996-12-24,-0.152396,-0.177929,-0.139649,-0.205399,-0.157531,-0.206993,-0.148729,-0.201520,-0.144172,-0.178814,-1.0,-1.0,-1.0,-1.0,1.0,-1.0
1996-12-26,-0.158155,-0.166752,-0.141000,-0.204704,-0.157174,-0.206428,-0.146865,-0.197090,-0.149051,-0.177158,1.0,-1.0,-1.0,-1.0,1.0,-1.0
1996-12-27,-0.149062,-0.161892,-0.142008,-0.203927,-0.156502,-0.205825,-0.144092,-0.193389,-0.155221,-0.178722,1.0,-1.0,-1.0,-1.0,1.0,-1.0
1996-12-30,-0.141697,-0.155962,-0.142937,-0.202710,-0.155261,-0.205482,-0.140937,-0.189108,-0.151205,-0.169736,1.0,-1.0,-1.0,-1.0,1.0,-1.0
1996-12-31,-0.135930,-0.165201,-0.143946,-0.201551,-0.154144,-0.205210,-0.140602,-0.186353,-0.147448,-0.165547,1.0,-1.0,-1.0,-1.0,1.0,-1.0


#### Create Loss and Benchmark data

In [105]:
# for the Benchmarks (BM1 and BM2) we will assume "always long in the market" i.e. signal = 1, always
df["BM1"] = -1 * 1 * df["sprd1"]
df["BM2"] = -1 * 1 * df["sprd2"]
df["loss1"] = -1 * df["signal1"] * df["sprd1"]
df["loss2"] = -1 * df["signal2"] * df["sprd2"]
df["loss3"] = -1 * df["signal3"] * df["sprd1"]
df["loss4"] = -1 * df["signal4"] * df["sprd2"]
df["loss5"] = -1 * df["signal5"] * df["sprd1"]
df["loss6"] = -1 * df["signal6"] * df["sprd2"]
df.tail()

,sprd1,sprd2,MAs1_12,MAs2_12,MAs1_8,MAs2_8,MAs1_4,MAs2_4,MAs1_1,MAs2_1,...,signal5,signal6,BM1,BM2,loss1,loss2,loss3,loss4,loss5,loss6
Date,,,,,,,,,,,,,,,,,,,,,
1996-12-24,-0.152396,-0.177929,-0.139649,-0.205399,-0.157531,-0.206993,-0.148729,-0.201520,-0.144172,-0.178814,...,1.0,-1.0,0.152396,0.177929,-0.152396,-0.177929,-0.152396,-0.177929,0.152396,-0.177929
1996-12-26,-0.158155,-0.166752,-0.141000,-0.204704,-0.157174,-0.206428,-0.146865,-0.197090,-0.149051,-0.177158,...,1.0,-1.0,0.158155,0.166752,0.158155,-0.166752,-0.158155,-0.166752,0.158155,-0.166752
1996-12-27,-0.149062,-0.161892,-0.142008,-0.203927,-0.156502,-0.205825,-0.144092,-0.193389,-0.155221,-0.178722,...,1.0,-1.0,0.149062,0.161892,0.149062,-0.161892,-0.149062,-0.161892,0.149062,-0.161892
1996-12-30,-0.141697,-0.155962,-0.142937,-0.202710,-0.155261,-0.205482,-0.140937,-0.189108,-0.151205,-0.169736,...,1.0,-1.0,0.141697,0.155962,0.141697,-0.155962,-0.141697,-0.155962,0.141697,-0.155962
1996-12-31,-0.135930,-0.165201,-0.143946,-0.201551,-0.154144,-0.205210,-0.140602,-0.186353,-0.147448,-0.165547,...,1.0,-1.0,0.135930,0.165201,0.135930,-0.165201,-0.135930,-0.165201,0.135930,-0.165201


#### Create the difference in performance vector

In [106]:
df_d = df[["loss1", "loss2", "loss3", "loss4", "loss5", "loss6", "BM1", "BM2"]]
df_d["d1"] = df_d["BM1"] - df_d["loss1"]
df_d["d2"] = df_d["BM2"] - df_d["loss2"]
df_d["d3"] = df_d["BM1"] - df_d["loss3"]
df_d["d4"] = df_d["BM2"] - df_d["loss4"]
df_d["d5"] = df_d["BM1"] - df_d["loss5"]
df_d["d6"] = df_d["BM2"] - df_d["loss6"]
df_d.tail()

C:\Users\cdbut\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\cdbut\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\cdbut\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,loss1,loss2,loss3,loss4,loss5,loss6,BM1,BM2,d1,d2,d3,d4,d5,d6
Date,,,,,,,,,,,,,,
1996-12-24,-0.152396,-0.177929,-0.152396,-0.177929,0.152396,-0.177929,0.152396,0.177929,0.304792,0.355859,0.304792,0.355859,0.0,0.355859
1996-12-26,0.158155,-0.166752,-0.158155,-0.166752,0.158155,-0.166752,0.158155,0.166752,0.000000,0.333503,0.316311,0.333503,0.0,0.333503
1996-12-27,0.149062,-0.161892,-0.149062,-0.161892,0.149062,-0.161892,0.149062,0.161892,0.000000,0.323784,0.298125,0.323784,0.0,0.323784
1996-12-30,0.141697,-0.155962,-0.141697,-0.155962,0.141697,-0.155962,0.141697,0.155962,0.000000,0.311924,0.283393,0.311924,0.0,0.311924
1996-12-31,0.135930,-0.165201,-0.135930,-0.165201,0.135930,-0.165201,0.135930,0.165201,0.000000,0.330401,0.271860,0.330401,0.0,0.330401


#### The Stationary Bootstrap

In [107]:
def stationaryBootstrap(numObs, numReps, blockSize):
    m = 1.0 / blockSize
    u = np.empty(numObs, dtype='int64')
    indices = np.empty((numReps, numObs), dtype='int64')

    for b in range(numReps):
        u[0] = np.random.randint(low=0, high=numObs, size=1, dtype='int64')
        v = np.random.uniform(size=numObs)

        for t in range(1, numObs):
            if v[t] < m:
                u[t] = np.random.randint(low=0, high=numObs, size=1, dtype='int64')
            else:
                u[t] = u[t-1] + 1
                if u[t] >= numObs: 
                    u[t] = u[t] - numObs

        indices[b] = u

    return indices

def tsbootstrap(y, numReps, blockSize):
    numObs = y.shape[0]
    ii = stationaryBootstrap(numObs, numReps, blockSize)
    yart = y[ii]
    
    return yart
    
## Main
yraw1 = df_d.d1
yraw2 = df_d.d2
yraw3 = df_d.d3
yraw4 = df_d.d4
yraw5 = df_d.d5
yraw6 = df_d.d6

numReps = 1000
blockSize = 10
values = np.empty(numReps)

dBoot1 = tsbootstrap(yraw1.values, numReps, blockSize)
dBoot2 = tsbootstrap(yraw2.values, numReps, blockSize)
dBoot3 = tsbootstrap(yraw3.values, numReps, blockSize)
dBoot4 = tsbootstrap(yraw4.values, numReps, blockSize)
dBoot5 = tsbootstrap(yraw5.values, numReps, blockSize)
dBoot6 = tsbootstrap(yraw6.values, numReps, blockSize)



dBoot6

array([[0.27996935, 0.26879344, 0.26911397, ..., 0.        , 0.        ,
        0.18030219],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.19359877, 0.        , 0.        , ..., 0.15740533, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.37876443, 0.36584902,
        0.34047364],
       [0.        , 0.        , 0.36867131, ..., 0.19113246, 0.13003101,
        0.11145815],
       [0.        , 0.34448914, 0.35561855, ..., 0.        , 0.        ,
        0.21583078]])

#### Create test statistics 

In [108]:
tRC_d1 = np.max(np.mean(yraw1.values))
tRC_d2 = np.max(np.mean(yraw2.values))
tRC_d3 = np.max(np.mean(yraw3.values))
tRC_d4 = np.max(np.mean(yraw4.values))
tRC_d5 = np.max(np.mean(yraw5.values))
tRC_d6 = np.max(np.mean(yraw6.values))

sub1 = dBoot1 - tRC_d1
sub2 = dBoot2 - tRC_d2
sub3 = dBoot3 - tRC_d3
sub4 = dBoot4 - tRC_d4
sub5 = dBoot5 - tRC_d5
sub6 = dBoot6 - tRC_d6

mBoot1 = sub1.mean(axis=1)
mBoot2 = sub2.mean(axis=1)
mBoot3 = sub3.mean(axis=1)
mBoot4 = sub4.mean(axis=1)
mBoot5 = sub5.mean(axis=1)
mBoot6 = sub6.mean(axis=1)


In [109]:
tRC_d1

0.1783024686122962

#### Create P-values and Results

In [110]:
p1 = np.empty(mBoot1.shape[0], dtype='int64')
p2 = np.empty(mBoot2.shape[0], dtype='int64')
p3 = np.empty(mBoot3.shape[0], dtype='int64')
p4 = np.empty(mBoot4.shape[0], dtype='int64')
p5 = np.empty(mBoot5.shape[0], dtype='int64')
p6 = np.empty(mBoot6.shape[0], dtype='int64')
for i in range(mBoot1.shape[0]):
    if mBoot1[i] > tRC_d1:
        p1[i] = 1
    else:
        p1[i] = 0

for i in range(mBoot2.shape[0]):
    if mBoot2[i] > tRC_d2:
        p2[i] = 1
    else:
        p2[i] = 0

for i in range(mBoot3.shape[0]):
    if mBoot3[i] > tRC_d3:
        p3[i] = 1
    else:
        p3[i] = 0

for i in range(mBoot4.shape[0]):
    if mBoot4[i] > tRC_d4:
        p4[i] = 1
    else:
        p4[i] = 0
        
for i in range(mBoot5.shape[0]):
    if mBoot5[i] > tRC_d5:
        p5[i] = 1
    else:
        p5[i] = 0
        
for i in range(mBoot6.shape[0]):
    if mBoot6[i] > tRC_d6:
        p6[i] = 1
    else:
        p6[i] = 0
    



In [111]:
Pvalue1 = np.mean(p1)
Pvalue2 = np.mean(p2)
Pvalue3 = np.mean(p3)
Pvalue4 = np.mean(p4)
Pvalue5 = np.mean(p5)
Pvalue6 = np.mean(p6)

# we will use and alpha = 0.05
def conclusion(pval, model):
    if pval < .05 and pval != 0.0:
        print("We reject the null hypothesis:", pval, "< .05")
        print("The p-value for",model,"is",pval)
    elif pval == 0.0:
        print("We FAIL to reject the null hypothesis since the p-value is exactly 0.0:")
        print("The p-value for",model,"is",pval)        
    else:
        print("We FAIL to reject the null hypothesis:")
        print("The p-value for",model,"is",pval)
        
m1 = "CL-GO MA(4,1)"
m2 = "CL-HO MA(4,1)"
m3 = "CL-GO MA(8,1)"
m4 = "CL-HO MA(8,1)"
m5 = "CL-GO MA(12,1)"
m6 = "CL-HO MA(12,1)"
conclusion(Pvalue1, m1)
print(" ")
conclusion(Pvalue2, m2)
print(" ")
conclusion(Pvalue3, m3)
print(" ")
conclusion(Pvalue4, m4)
print(" ")
conclusion(Pvalue5, m5)
print(" ")
conclusion(Pvalue6, m6)

We FAIL to reject the null hypothesis since the p-value is exactly 0.0:
The p-value for CL-GO MA(4,1) is 0.0
 
We FAIL to reject the null hypothesis since the p-value is exactly 0.0:
The p-value for CL-HO MA(4,1) is 0.0
 
We FAIL to reject the null hypothesis since the p-value is exactly 0.0:
The p-value for CL-GO MA(8,1) is 0.0
 
We FAIL to reject the null hypothesis since the p-value is exactly 0.0:
The p-value for CL-HO MA(8,1) is 0.0
 
We FAIL to reject the null hypothesis since the p-value is exactly 0.0:
The p-value for CL-GO MA(12,1) is 0.0
 
We FAIL to reject the null hypothesis since the p-value is exactly 0.0:
The p-value for CL-HO MA(12,1) is 0.0


## THE END